In [112]:
import os
import pandas as pd
import datetime

desktop = 'c:/users/geoffrey/desktop/'
filename = 'nba.csv'
output_filename =  'upcoming_playoff_games.csv'

months_list=[]
for i in [4,5]: # only want to download april + may
    months_list.append( datetime.date(2018,i,1).strftime('%B').lower() )
months_list

# goal is to find upcoming NBA playoff schedule the hard way
# scrape all games played in april/may, clean the data
# then export games with no scores, those being the upcoming, unplayed games


['april', 'may']

In [113]:
df=pd.DataFrame()

# scrape NBA games from website, save to csv on desktop
# read csv file and begin to clean it up.
if not os.path.exists(desktop+filename):
    dl = []
    for month in months_list:
        dd=pd.read_html("http://www.basketball-reference.com/leagues/NBA_2018_games-{}.html".format(month))
        dl.append(dd[0])
    df = pd.concat ( dl[:], ignore_index=True, axis=0 )
    df.to_csv(desktop+filename)
else:
    df = pd.read_csv(desktop+filename)
    
print(df.head())


               Date Start (ET)     Visitor/Neutral    PTS  \
0  Sun, Apr 1, 2018    6:00 pm       Orlando Magic   88.0   
1  Sun, Apr 1, 2018    6:00 pm     Detroit Pistons  108.0   
2  Sun, Apr 1, 2018    3:30 pm  Washington Wizards   94.0   
3  Sun, Apr 1, 2018    1:00 pm  Philadelphia 76ers  119.0   
4  Sun, Apr 1, 2018    6:00 pm    Dallas Mavericks   87.0   

          Home/Neutral  PTS.1              .1  Attend.  Notes  
0        Atlanta Hawks   94.0  Box Score  NaN  13587.0    NaN  
1        Brooklyn Nets   96.0  Box Score  NaN  16097.0    NaN  
2        Chicago Bulls  113.0  Box Score  NaN  20466.0    NaN  
3    Charlotte Hornets  102.0  Box Score  NaN  17005.0    NaN  
4  Cleveland Cavaliers   98.0  Box Score  NaN  20562.0    NaN  


In [114]:
# look through the columns to see what needs cleaning
# What's in that column ".1" ?
print( df.iloc[:,7].value_counts() )
print( df.info() )

OT    8
Name:  .1, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 10 columns):
Date               159 non-null object
Start (ET)         153 non-null object
Visitor/Neutral    158 non-null object
PTS                149 non-null float64
Home/Neutral       158 non-null object
PTS.1              149 non-null float64
                   149 non-null object
 .1                8 non-null object
Attend.            149 non-null float64
Notes              0 non-null float64
dtypes: float64(4), object(6)
memory usage: 8.7+ KB
None


In [115]:
# rename columns
headers = list(df.columns)
column_names = ['date', 'tipoff', 'visiting_team', 'visiting_points', 'home_team', 'home_points', 'html_link', 'overtime', 'attendance', 'notes']
header_dictionary = dict(zip(headers, column_names))

date_format='%a, %b %d, %Y'
tipoff_format = '%-I:%M %p'

In [116]:
# here's our main cleaning script

def process_df(df):
    clean_df = (df.rename(columns=header_dictionary)
            .assign(date=lambda x: pd.to_datetime(x['date'], format=date_format, errors='coerce')
                    ,tipoff=lambda x: pd.to_datetime(x['tipoff'], format=tipoff_format, errors='coerce')
                   )
            .dropna(thresh=4)
            .drop(['html_link','overtime','notes'], axis=1)
            .set_index('date')
           )
    return clean_df

clean_df = process_df(df)
print(clean_df.info())
clean_df.sample(10)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 153 entries, 2018-04-01 to 2018-05-11
Data columns (total 6 columns):
tipoff             153 non-null object
visiting_team      153 non-null object
visiting_points    149 non-null float64
home_team          153 non-null object
home_points        149 non-null float64
attendance         149 non-null float64
dtypes: float64(3), object(3)
memory usage: 6.6+ KB
None


,tipoff,visiting_team,visiting_points,home_team,home_points,attendance
date,,,,,,
2018-04-01,8:30 pm,Phoenix Suns,107.0,Golden State Warriors,117.0,19596.0
2018-04-27,8:00 pm,Cleveland Cavaliers,87.0,Indiana Pacers,121.0,17923.0
2018-04-30,8:00 pm,Philadelphia 76ers,101.0,Boston Celtics,117.0,18624.0
2018-05-08,10:30 pm,New Orleans Pelicans,NaN,Golden State Warriors,NaN,NaN
2018-04-14,10:30 pm,New Orleans Pelicans,97.0,Portland Trail Blazers,95.0,19882.0
2018-04-17,10:30 pm,New Orleans Pelicans,111.0,Portland Trail Blazers,102.0,20066.0
2018-04-07,8:30 pm,Oklahoma City Thunder,108.0,Houston Rockets,102.0,18055.0
2018-05-01,8:00 pm,Cleveland Cavaliers,113.0,Toronto Raptors,112.0,19954.0
2018-04-18,7:00 pm,Indiana Pacers,97.0,Cleveland Cavaliers,100.0,20562.0


In [117]:
# export to desktop as 'playoffs.csv'

playoffs = clean_df[clean_df.visiting_points.isnull()]
playoffs.to_csv(desktop+output_filename)
playoffs

,tipoff,visiting_team,visiting_points,home_team,home_points,attendance
date,,,,,,
2018-05-08,10:30 pm,New Orleans Pelicans,NaN,Golden State Warriors,NaN,NaN
2018-05-08,8:00 pm,Utah Jazz,NaN,Houston Rockets,NaN,NaN
2018-05-09,8:00 pm,Philadelphia 76ers,NaN,Boston Celtics,NaN,NaN
2018-05-11,8:00 pm,Boston Celtics,NaN,Philadelphia 76ers,NaN,NaN
